In [ ]:
# load all required libraries for Dog's Breed Identification Project
import cv2
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input

#read the csv file
df_labels = pd.read_csv("labels.csv")

#store training and testing images folder location
train_file = 'train/'

#check the total number of unique breed in our dataset file
print("Total number of unique Dog Breeds :",len(df_labels.breed.unique()))

#specify number
num_breeds = 60
im_size = 224
batch_size = 64
encoder = LabelEncoder()

#get only 60 unique breeds record 
breed_dict = list(df_labels['breed'].value_counts().keys()) 
new_list = sorted(breed_dict,reverse=True)[:num_breeds*2+1:2]

#change the dataset to have only those 60 unique breed records
df_labels = df_labels.query('breed in @new_list')

#create new column which will contain image name with the image extension
df_labels['img_file'] = df_labels['id'].apply(lambda x: x + ".jpg")

#create a numpy array of the shape
train_x = np.zeros((len(df_labels), im_size, im_size, 3), dtype='float32')
 
#iterate over img_file column of our dataset
for i, img_id in enumerate(df_labels['img_file']):
  img = cv2.resize(cv2.imread(train_file+img_id,cv2.IMREAD_COLOR),((im_size,im_size)))
  img_array = preprocess_input(np.expand_dims(np.array(img[...,::-1].astype(np.float32)).copy(), axis=0))
  train_x[i] = img_array

train_y = encoder.fit_transform(df_labels["breed"].values)

x_train, x_test, y_train, y_test = train_test_split(train_x,train_y,test_size=0.2,random_state=42)

#Image augmentation using ImageDataGenerator class
train_datagen = ImageDataGenerator(rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
 
#generate images for training sets 
train_generator = train_datagen.flow(x_train, 
                                     y_train, 
                                     batch_size=batch_size)
 
#same process for Testing sets also by declaring the instance
test_datagen = ImageDataGenerator()
 
test_generator = test_datagen.flow(x_test, 
                                     y_test, 
                                     batch_size=batch_size)

#building the model using ResNet50V2 with input shape of our image array
resnet = ResNet50V2(input_shape = [im_size,im_size,3], weights='imagenet', include_top=False)

#freeze all trainable layers and train only top layers 
for layer in resnet.layers:
    layer.trainable = False

#add global average pooling layer and Batch Normalization layer
x = resnet.output
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)

#add fully connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

#add output layer having the shape equal to number of breeds
predictions = Dense(num_breeds, activation='softmax')(x)
 
#create model class with inputs and outputs
model = Model(inputs=resnet.input, outputs=predictions)

#epochs for model training and learning rate for optimizer
epochs = 20
learning_rate = 1e-3

#using RMSprop optimizer compile or build the model
optimizer = RMSprop(learning_rate=learning_rate,rho=0.9)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

# fit the training generator data and train the model
hist = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,  
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.n // batch_size,  
    verbose=1  
)

#Save the model for prediction
model.save("model.keras")

# Print all the unique dog breeds in sorted order
unique_breeds_sorted = sorted(df_labels['breed'].unique())
print(unique_breeds_sorted)

# Load the model
model = load_model("model.keras")

# Get the image of the dog for prediction
pred_img_path = r"C:\Users\Dell\Desktop\Dog Breed Identification\dog_breed_images\d20.png"

# Read the image separately
img = cv2.imread(pred_img_path, cv2.IMREAD_COLOR)

# Resize the image for model input
pred_img_array = cv2.resize(img, (im_size, im_size))

# Preprocess the image
pred_img_array = preprocess_input(np.expand_dims(np.array(pred_img_array[...,::-1].astype(np.float32)).copy(), axis=0))

# Predict
pred_val = model.predict(np.array(pred_img_array, dtype="float32"))

# Get the breed
pred_breed = sorted(new_list)[np.argmax(pred_val)]

# Print the predicted breed
print("Predicted Breed for this Dog is:", pred_breed)

# Showing the image and converting BGR (OpenCV format) to RGB for correct color display
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Plotting
plt.figure(figsize=(6,6))
plt.imshow(img_rgb)
plt.axis('off')
plt.title(f"Predicted Breed: {pred_breed}", fontsize=15)
plt.show()
